# Tic-Tac-Toe

In [47]:
class TicTacToe:
    """ Tic-Tac Toe Board 
        
        Board is represented through a board object
        
        0 ¦ 1 ¦ 2
        --+---+--
        3 ¦ 4 ¦ 5
        --+---+--
        6 ¦ 7 ¦ 8
    """
    def __init__(self):
        self.spaces = 9
        self.blank = " "
        self.legal_players = ["X","O"]
        self.board = [" "] * self.spaces
        self.base =      " {} ¦ {} ¦ {} \n" + \
                          "---+---+--- \n" + \
                         " {} ¦ {} ¦ {} \n" + \
                          "---+---+--- \n" + \
                         " {} ¦ {} ¦ {} \n" 
        self.winning = [[0,1,2], # top horizontal
                        [3,4,5], # middle horizontal
                        [6,7,8], # bottom horizontal
                        [0,3,6], # left vertical
                        [1,4,7], # middle vertical
                        [2,5,8], # right vertical
                        [0,4,8], # diagonal \
                        [2,4,6]  # diagonal /
                       ]
        
    def __str__(self):
        """ String representation of a game """
        return self.base.format(*self.board)
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return self.board == other.board
        return False

    def __hash__(self):
        return hash("".join(self.board))
    
    def copy(self):
        """ Creates a copy of the game object """
        new_board = self.__class__()
        new_board.board = self.board[:]
        return new_board
        
    def get_moves(self):
        """ Returns open moves on the board """
        return [idx for idx, player in enumerate(self.board) if player == self.blank]
    
    def move(self, player, move):
        """ Updates the board based on player and move 
            
            example: 
                game.move("X", 4)                
        """
        player = player.upper()
        assert move in self.get_moves(), "Illegal move"
        assert player in self.legal_players, "Illegal player"
        self.board[move] = player
        

    def gameover(self):
        """ Returns whether there are any legal moves left """
        return len(self.get_moves()) == 0 or self.winner() is not None
    
    def winner(self):
        """ Returns the winner of a game, 'Draw' or None if game is ongoing """
        for winner in self.winning:
            players = [self.board[pos] for pos in winner]
            s_players = set(players)
            if len(s_players) == 1 and self.blank not in s_players:
                return players[0]
        
        if self.blank not in set(self.board):
            return "Draw"

    def score_game(self, player):
        """ Scores a game based relative to player provided """
        winner = self.winner()
        player = player.upper()
        alt_player = "X" if player == "O" else "O"

        if winner == player:
            return 1
        elif winner == alt_player:
            return -1
        return 0

In [48]:
best_moves = {}

def get_best_move(game, player="X"):
    """ Returns best move given game and player """
    return mini_max(game,player)[0]

def mini_max(game, player="X"):
    """ Helper function for get_best_move. Returns best move and score given game and player """
    if player not in best_moves:
        # For memoization. If best_moves dictionary not initalized for current player, initiliaze
        best_moves[player] = {}

    best_moves_player = best_moves[player] # Get best moves for player
    
    if game not in best_moves_player:
        # Check to see if we've already seen this state
        if game.gameover():
            # Game over, no moves possible
            best_move = None
            best_score = game.score_game(player)
        else:
            alt_player = "O" if player == "X" else "X" 
            moves = game.get_moves() # Get available moves
            best_score = float("-inf") # Default to worst possile score to ensure any move is selected

            for move in moves:
                clone = game.copy() # Make a copy of the game to run recursively
                clone.move(player, move) # Make the proposed move
                _, score = mini_max(clone, player=alt_player) # Figure out what the opponents best move would be (MINI)
                score *= -1 # Since the game is zero-sum, what's bad for the opponent is good for us
                if score > best_score: # Best our prior best score
                    best_move = move # Save the move
                    best_score = score # Update the best score
        best_moves[player][game] = (best_move, best_score) # Update the best move and score given the game
    return best_moves[player][game] # Return best move given player and game

In [51]:
def play():
    human = None
    while human is None:
        human = input("Play as X or O? ").upper()
        if human not in ["X","O"]:
            print("Invalid option")
            human = None
            
    comp = "X" if human == "O" else "O"
    game = TicTacToe()
    turn = "X"
    while not game.gameover():    
        if comp == turn:
            move = get_best_move(game, comp)
        else:
            print(game)
            move = None
            while move is None:
                move = input("Input move: ")
                if move.upper()[:1] == "Q":
                    print("Quitter...")
                    return
                elif move.upper()[:1] == "H":
                    print(game.__doc__)
                    continue
                if move.isdigit():
                    move = int(move)
        if move in game.get_moves():
            game.move(turn, move)
            turn = "X" if turn == "O" else "O"
        else:
            print("Illegal move. Q to quit. H for help")
    print(game)
    if game.winner() == human:
        print("You won!")
    elif game.winner() == comp:
        print("You lost :-(")
    else:
        print("It's a draw")
    

In [52]:
play()

Play as X or O? o
 X ¦   ¦   
---+---+--- 
   ¦   ¦   
---+---+--- 
   ¦   ¦   

Input move: 1
 X ¦ O ¦   
---+---+--- 
 X ¦   ¦   
---+---+--- 
   ¦   ¦   

Input move: 5
 X ¦ O ¦   
---+---+--- 
 X ¦ X ¦ O 
---+---+--- 
   ¦   ¦   

Input move: 6
 X ¦ O ¦   
---+---+--- 
 X ¦ X ¦ O 
---+---+--- 
 O ¦   ¦ X 

You lost :-(
